In [ ]:
!pip install stegano

In [ ]:
import os
from stegano import lsb #USED FOR PNG IMAGE
import skimage.io as sk
import matplotlib.pyplot as plt
from scipy import spatial
from tqdm import tqdm

from PIL import Image
from random import shuffle


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# /kaggle/input/alaska2-image-steganalysis/sample_submission.csv
# /kaggle/input/alaska2-image-steganalysis/Cover/64601.jpg

In [ ]:
image = sk.imread("/kaggle/input/alaska2-image-steganalysis/Cover/64601.jpg")

secret = lsb.hide("/kaggle/input/alaska2-image-steganalysis/Cover/64601.jpg", "duoduo")
secret.save("encoded.png")

In [ ]:
img1 = sk.imread("/kaggle/input/alaska2-image-steganalysis/Cover/64601.jpg")
img2 = sk.imread("/kaggle/working/encoded.png")

fig,ax = plt.subplots(1,2,figsize=(18,8))
    
ax[0].imshow(img1)
ax[1].imshow(img2)

In [ ]:
print(lsb.reveal("/kaggle/working/encoded.png"))

(img1 - img2).sum()

In [ ]:
# Knaive
vec1 = np.reshape(img1,(512*512*3))
vec2 = np.reshape(img2,(512*512*3))

# cosine similarity
print(spatial.distance.cosine(vec1,vec2))
print(1 - spatial.distance.cosine(vec1,vec2))

### Data exploration

In [ ]:
BASE_PATH = "/kaggle/input/alaska2-image-steganalysis"
train_imageids = pd.Series(os.listdir(BASE_PATH + '/Cover')).sort_values(ascending=True).reset_index(drop=True)
test_imageids = pd.Series(os.listdir(BASE_PATH + '/Test')).sort_values(ascending=True).reset_index(drop=True)

train_imageids

In [ ]:
cover_images_path = pd.Series(BASE_PATH + '/Cover/' + train_imageids ).sort_values(ascending=True)
JMIPOD_images_path = pd.Series(BASE_PATH + '/JMiPOD/'+train_imageids).sort_values(ascending=True)
JUNIWARD_images_path = pd.Series(BASE_PATH + '/JUNIWARD/'+train_imageids).sort_values(ascending=True)
UERD_images_path = pd.Series(BASE_PATH + '/UERD/'+train_imageids).sort_values(ascending=True)
test_images_path = pd.Series(BASE_PATH + '/Test/'+test_imageids).sort_values(ascending=True)
ss = pd.read_csv(f'{BASE_PATH}/sample_submission.csv')

ss

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(30, 25))
k=0
for i, row in enumerate(ax):
    for j, col in enumerate(row):
        img = sk.imread(cover_images_path[k])
        col.imshow(img)
        col.set_title(cover_images_path[k])
        k=k+1
plt.suptitle('Cover 图', fontsize=14)
plt.show()

In [ ]:
# 展示cover 加 三种编码的 stego
fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(30, 15))
for i in range(3):
    cvimg = sk.imread(cover_images_path[i])
    uniimg = sk.imread(JUNIWARD_images_path[i])
    jpodimg = sk.imread(JMIPOD_images_path[i])
    uerdimg = sk.imread(UERD_images_path[i])
    
    ax[i,0].imshow(cvimg)
    ax[i,0].set_title('Cover_IMG'+train_imageids[i])
    ax[i,1].imshow(uniimg)
    ax[i,1].set_title('JNIWARD_IMG'+train_imageids[i])
    ax[i,2].imshow(jpodimg)
    ax[i,2].set_title('JMiPOD_IMG'+train_imageids[i])
    ax[i,3].imshow(uerdimg)
    ax[i,3].set_title('UERD_IMG'+train_imageids[i])

In [ ]:
# 查看隐写区域
img_cover = sk.imread(cover_images_path[0])
img_jmipod = sk.imread(JMIPOD_images_path[0])
img_juniward = sk.imread(JUNIWARD_images_path[0])
img_uerd = sk.imread(UERD_images_path[0])

fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(16, 12))

ax[0,1].set_title(f'{train_imageids[k]} Channel 0')
ax[0,2].set_title(f'{train_imageids[k]} Channel 1')
ax[0,3].set_title(f'{train_imageids[k]} Channel 2')

ax[0,0].imshow(img_jmipod)
ax[0,1].imshow((img_cover == img_jmipod).astype(int)[:,:,0])
ax[0,2].imshow((img_cover == img_jmipod).astype(int)[:,:,1])
ax[0,3].imshow((img_cover == img_jmipod).astype(int)[:,:,2])
ax[0,0].set_ylabel('JMiPOD', rotation=90, size='large')

ax[1,0].imshow(img_juniward)
ax[1,1].imshow((img_cover == img_juniward).astype(int)[:,:,0])
ax[1,2].imshow((img_cover == img_juniward).astype(int)[:,:,1])
ax[1,3].imshow((img_cover == img_juniward).astype(int)[:,:,2])
ax[1,0].set_ylabel('JUNIWARD', rotation=90, size='large')

ax[2,0].imshow(img_uerd)
ax[2,1].imshow((img_cover == img_uerd).astype(int)[:,:,0])
ax[2,2].imshow((img_cover == img_uerd).astype(int)[:,:,1])
ax[2,3].imshow((img_cover == img_uerd).astype(int)[:,:,2])
ax[2,0].set_ylabel('UERD', rotation=90, size='large')

plt.suptitle('Pixel Deviation from Cover Image')
plt.show()

(img_cover == img_juniward).shape

In [ ]:
# 在YCbCr颜色空间上查看图像的三个通道 Cover
fig,ax = plt.subplots(3,4,figsize=(20,16))

for i,paths in enumerate(cover_images_path[:3]):
    image = Image.open(paths)
    ycbcr = image.convert('YCbCr')
    (y, cb, cr) = ycbcr.split()

    ax[i,0].imshow(image)
    ax[i,0].set_title('Cover'+train_imageids[i])
    ax[i,1].imshow(y)
    ax[i,1].set_title('Y : Luminance')
    ax[i,2].imshow(cb)
    ax[i,2].set_title('Cb : Chroma Blue')
    ax[i,3].imshow(cr)
    ax[i,3].set_title('Cr : Chroma Red')

In [ ]:
# 查看三种隐写的 YCbCr 通道
fig,ax = plt.subplots(4,4,figsize=(20,16))
plt.tight_layout()


im1 = Image.open(cover_images_path[0])
im2 = Image.open(JUNIWARD_images_path[0])
im3 = Image.open(JMIPOD_images_path[0])
im4 = Image.open(UERD_images_path[0])

for i,image in enumerate([im1,im2,im3,im4]):
    ycbcr = image.convert('YCbCr')
    (y, cb, cr) = ycbcr.split()

    ax[i,0].imshow(image)
    ax[i,0].set_title('Image')
    ax[i,1].imshow(y)
    ax[i,1].set_title('Luminance')
    ax[i,2].imshow(cb)
    ax[i,2].set_title('Cb:Chroma Blue')
    ax[i,3].imshow(cr)
    ax[i,3].set_title('Cr:Chroma Red')

In [ ]:
! git clone https://github.com/dwgoon/jpegio
!pip install jpegio/.
import jpegio as jio

In [ ]:
# 查看三个通道的DCT系数
fig,ax = plt.subplots(4,4,figsize=(20,16))
plt.tight_layout()

for i,path in enumerate([cover_images_path[0],JUNIWARD_images_path[0],JMIPOD_images_path[0],UERD_images_path[0]]):
    
    image = Image.open(path)
    jpeg = jio.read(path)
    DCT_Y = jpeg.coef_arrays[0]
    DCT_Cr = jpeg.coef_arrays[1]
    DCT_Cb = jpeg.coef_arrays[2]
    
    
    ax[i,0].imshow(image)
    ax[i,0].set_title('Image')
    ax[i,1].imshow(DCT_Y)
    ax[i,1].set_title('Luminance')
    ax[i,2].imshow(DCT_Cb)
    ax[i,2].set_title('Cb:Chroma Blue')
    ax[i,3].imshow(DCT_Cr)
    ax[i,3].set_title('Cr:Chroma Red')

In [ ]:
# 查看DCT系数区别
coverDCT = np.zeros([512,512,3])
stegoDCT = np.zeros([512,512,3])
jpeg = jio.read(cover_images_path[0])
stego_juni = jio.read(JUNIWARD_images_path[0])

coverDCT[:,:,0] = jpeg.coef_arrays[0]
coverDCT[:,:,1] = jpeg.coef_arrays[1]
coverDCT[:,:,2] = jpeg.coef_arrays[2]
stegoDCT[:,:,0] = stego_juni.coef_arrays[0]
stegoDCT[:,:,1] = stego_juni.coef_arrays[1]
stegoDCT[:,:,2] = stego_juni.coef_arrays[2]

DCT_diff = coverDCT - stegoDCT
print("DCT系数相异数目：",len(DCT_diff[np.where(DCT_diff!=0)]))
print("DCT差异量：",np.unique(DCT_diff))
plt.figure(figsize=(16,10))
plt.imshow(abs(DCT_diff) )
plt.show()

In [ ]:
# 从像素入手查看差别
coverPixels = np.array(Image.open(cover_images_path[0])).astype('float')
stegoPixels = np.array(Image.open(JUNIWARD_images_path[0])).astype('float')

pixelsDiff = coverPixels - stegoPixels

print(len(pixelsDiff[np.where(pixelsDiff!=0)]))
print(np.unique(pixelsDiff))
plt.figure(figsize=(16,10))
plt.imshow(abs(pixelsDiff))
plt.show()

In [ ]:
# 对比两个 diff 图
fig,ax = plt.subplots(1,2,figsize=(16,12))
ax[0].imshow(abs(DCT_diff))
ax[1].imshow(abs(pixelsDiff))